

databricks workflow jobs
  bascially azure datafactory helps to automate all the tasks 
  task orchestration 


optimized storage
  Delta lake
  parquet
  ice berg

Unity Catalog
  is a centralized data catalog that provides access control, auditing, data lineage, quality monitoring and data discovery across Databricks workspaces.

Lake Flow
  Connect :- A set of efficient ingestion connectors that simplify data ingestion from popular applications,database,cloud storage, message buses and local files
  
  Declarative pipelines :- A framework for building batch and streaming data pipelines using sql and python, desgined to accelerate ETL development.
  
  Jobs :- A workflow automation tool for Databricks that orchestrates data processing workloads.



Data ingestion with LakeFlow connect
  Data ingestion is streamlined with simple, efficient connectors that enable you to bring in data from files, cloud storage, databases, enterprise applications and streaming sources directly inyo Data bricks Lakehouse- all within a unified, managed platform






Data ingestion from cloud
  Goal is to convert raw files from cloud into delta tables in databricks
  Ingestion from cloud object storage is performed using Lakeflow connect standard connectors

  1.create table as (ctas)(batch read) :- creates a delta table by default from the files stored in cloud object.
      read_files :- 
        1. function is used to read files from a specified location and return the data in a tabular format.
        2. automatically detcts the file format and infers a unified schema across all files.
        3. allows you to specify format-specify options for greater control when reading source files.
        4. can be used in streaming tables to incrementally ingest files into delta lake using auto loader.
  
    format below
    create table new_table_name as 
      select * from 
        read_files(<path_to_files>,
                    format => '<file_type>',
                    <other_format_specific_options>
        );

  2.copy into(incremental load)
      This command performs a bulk load from files in cloud object storage into the table, in the below example, it will load files into the empty table new_table. the FROM clause specifies the location of the csv files.
      Create an empty table to copy data into.
        create an empty table without a schema/ explicity create the table with a schema
      
      COPY INTO (legacy)
      ******************
      CREATE TABLE new_table;

      COPY INTO new_table
      FROM '<dir_path>'
      FILEFORMAT=<file_type>
      FORMAT_OPTIONS(<options>)
      COPY_OPTIONS(<options>)


      1.is a retriable and idempotent operation and will skip files that have already been loaded(incremental)
      2.supports various common files types like parquet, json, xml etc.
      3.FROM specifies the path of the cloud storage location continuously adding files
      4.FORMAT_OPTIONS() controls how the source files are parsed and interpreted,
      5.COPY_OPTIONS() controls the behviour of the COPY INTO operation itself, such as schema ec=vloution(mergeSchema) and idemptency (force)


  3.Auto loader
      1.incrementally and efficiently processes new data files(in batch or streaming) as they arrive in cloud storage without any additional setup
      2.can use auto loader to process billions of files
      3.Auto loader is built upon Spark Structured Streaming
    
    Python Auto loader

      spark
        .readStream
          .format("cloudFiles")
          .option("cloudFiles.format","json")
          .option("cloudFiles.schemaLocation","<checkpoint_path>")
          .load("/Volumes/catalog/schema/files")
        .writeStream
          .option("checkpointLocation","<checkpoint_path>")
          .trigger(processingTime="5 seconds")
          .toTable("catalog.database.table")
    
    SQL Auto loader

      CREATE or REFRESH STREAMING TABLE
      catalog.schema.table SCHEDULE EVERY 1 HOUR
      AS
      SELECT * 
      FROM STREAM read_files(
        '<dir_path>',
        format => '<file_type>'
      )



    Refresh Streaming table(manually refresh table)
      REFRESH STREAMING TABLE catalog.schema.table;




Rescued Data Column
  captures mismatched or unparsable fields as json. preserving non-conforming input values in your lakehouse tables instead of dropping them.

  read_files(),spark.read or aut loader provides a rescued data column if the raw data doesnt not match the schema.



Lakeflow Connect(managed connectors)
***********************************
    data ingestion from wide variety of enterprise databases

    lakeflow connect collects data from external sources to streaming delta tables using a serverless compute declarative pipelines

    pipeline (Sas or api)
      1. A lakeflow serverless declarative pipeline job collects credentials from unity catalog
      2. the job reaches out to the publicly accessible data source(eg. API, open OLAP port etc..)
      3. The service transforms the data and stores it to a streaming delta table


    Lakeflow connect collects data from external databases to streaming delta tables
      1. The classic compute declartive pipelines job collects credentials from unity catalog(uc)
      2. It uses the credentials to connect and collect data from your databases surces
      3. The latest state and staging data are saved to your unity catalog volume
      4. A serverless declartive pipeline job process the collected data to your streaming delta table


      A dedicated pipeline that connects to the database to extract
        metadata
        snapshots
        change logs - it stages all of this in the unity catalog(UC) volume
      
      unity catalog volume:
        This acts as the intermediate staging layer, enabling the next piepline to pick up and stream data 
        Its secured using standard UC mechanics, and by default, access is liited to the user running pipeline
  




  ingesting into existing delta table
      MERGE INTO target_table target
      USING source_table source
      on target.id = source.id



      update
      
      MERGE INTO target_table target
      USING source_table source
      on target.id = source.id
      WHEN MATCHED AND source.status = 'update' THEN
        UPDATE SET
          target.email = source.email,
          target.status = source.status
      WHEN MATCHED AND source.status = 'delete' THEN
        DELETE
      WHEN NOT MATCHED THEN
        INSERT (id,first_name,email,sign_up_date,status)
        VALUES(source.id,source.first_name,source.email,source.sign_up_date,source.status);












Jobs
  primary resource for 
    1. Scheduling
    2. Coordinating
    3. Running operations such as data processing, ETL, analytics and machine learning workloads within databricks environment

task
  single unit of work wirhin a job that executes a specific workload suck as notebook, script, query and more


Each job consists of one or more tasks, which are the individual units of work that make up the job





Change data capture (CDC)
  is a tecnique used to track and capture changes in a data source (such as database, lakehouse or data warehouse)
  Those changes are then applied to a target table, for example your lakehouse, to keeo it up to date with the latest state from the source.

  CDC is also closely related to how we handle slowly changing dimensions, or SCDs, which define how histroical changes are tracked and stored in your target

    SCD Type 1 :- Overwrites existing data (no history tracking)
    SCD Type 2 :- Tracks historical changes by storing previous versions of records

  
  Using APPLY CHANGES INTO to perform SCD in Declarative pipelines(INSERTs and UPDATEs are implicitly implemented using the KEY(s), no coding required)

APPLY CHANGES INTO target_table
FROM STREAM source_table
KEYS (column) --The column(s) that uniquely identify a row in the source and target table(primary )
APPLY AS DELETE WHEN operation = "DELETE" --Specifies whena cdc event should be treated asa DELETE rather thananupsert
SEQUENCE BY Column_name -- specifies the logical order of cdc events in spurce data
COLUMNS * EXCEPT (operation) -- specifes a subset of columns to include in the target
STORED AS SCD TYPE 2; -- whether to store records as SCD TYPE 1 (default) or TYPE 2











REPO

Git:
  new repository (https://github.com/mail4cirus/databricks_training_2025.git)



Databricks:
  create git folder using git url(workspace -> user -> user@id -> create -> git folder)
  integrate (databricks + git) (settings -> linked accounts => github -> configure with the branch name)











Streaming
***********
1. Structure Streaming